In [14]:
from flask import Flask
from flask_restful import reqparse, abort, Api, Resource
import pickle
import numpy as np
from model import NLPModel

app = Flask(__name__)
api = Api(app)# create new model object
model = NLPModel()

# load trained classifier
clf_path = 'lib/models/SentimentClassifier.pkl'
with open(clf_path, 'rb') as f:
    model.clf = pickle.load(f)# load trained vectorizer
    
vec_path = 'lib/models/TFIDFVectorizer.pkl'
with open(vec_path, 'rb') as f:
    model.vectorizer = pickle.load(f)
    
# argument parsing
parser = reqparse.RequestParser()
parser.add_argument('query')

class PredictSentiment(Resource):
        def get(self):
            # use parser to find the user's query
            args = parser.parse_args()
            user_query = args['query']
            
            # vectorize the user's query and make a prediction
            uq_vectorized = model.vectorizer_transform(
                np.array([user_query]))
            prediction = model.predict(uq_vectorized)
            pred_proba = model.predict_proba(uq_vectorized)
            
            # Output 'Negative' or 'Positive' along with the score
            if prediction == 0:
                pred_text = 'Negative'
            else:
                pred_text = 'Positive'
            
            # round the predict proba value and set to new variable
            confidence = round(pred_proba[0], 3)
            
            # create JSON Object
            output = {'prediction': pred_text, 'confidence': confidence}
            
            return output
        

api.add_resource(PredictSentiment, '/')
  
# example of another endpoint
#api.add_resource(PredictRatings, '/ratings')

if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


OSError: [Errno 48] Address already in use